In [1]:
import numpy as np
import pandas as pd
import mediapipe as mp
import os
import cv2


In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
mp_holistic = mp.solutions.holistic #Holistic model 
mp_drawing = mp.solutions.drawing_utils #drawing utiliites

In [3]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    #                             mp_drawing.DrawingSpec(color=(80,110,10),thickness = 1, circle_radius=1),
    #                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
    #                          )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10),thickness = 2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76),thickness = 2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,120,60),thickness = 2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(245,56,221),thickness=2,circle_radius=2))


In [5]:
def extract_keypoints(results):
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)

    # pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # pose = pose[:40]
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([ lh, rh])


In [6]:
IMAGES_PATH  = 'Data/collectedimages'
KEYPOINTS_PATH = 'Data/colectedkeypoints'
HolisticImages_PATH = 'Data/HolisticImages'

In [7]:
if not os.path.exists(HolisticImages_PATH):
    os.makedirs(HolisticImages_PATH)
if not os.path.exists(KEYPOINTS_PATH):
    os.makedirs(KEYPOINTS_PATH)

In [8]:
label_map = os.listdir(IMAGES_PATH)

In [9]:
for label in label_map:
    if not os.path.exists(os.path.join(KEYPOINTS_PATH, label)):
        os.makedirs(os.path.join(KEYPOINTS_PATH, label))
    if not os.path.exists(os.path.join(HolisticImages_PATH, label)):
        os.makedirs(os.path.join(HolisticImages_PATH, label))
    image_dir = os.path.join(IMAGES_PATH, label)
    num_images = len(os.listdir(image_dir))
    for i, image_path in enumerate(os.listdir(image_dir)):
        image = cv2.imread(os.path.join(image_dir, image_path))
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            image, results = mediapipe_detection(image, holistic)
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(KEYPOINTS_PATH, label, image_path.replace('.jpg', ''))
            np.save(npy_path, keypoints)
            draw_styled_landmarks(image, results)
            cv2.imwrite(os.path.join(HolisticImages_PATH, label, image_path), image)

I0000 00:00:1731067458.986785   11977 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1731067458.989290   16475 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1731067459.077272   16460 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731067459.108431   16459 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731067459.111575   16460 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731067459.111705   16458 inference_feedback_manager.cc:114] Feedback manager requires a model with a single 

In [10]:
def load_keypoints(label):
    label_path = os.path.join(KEYPOINTS_PATH, label)
    keypoints_list = []

    # Duyệt qua từng tệp trong thư mục nhãn
    for file in os.listdir(label_path):
        if file.endswith('.npy'):
            keypoints = np.load(os.path.join(label_path, file))
            keypoints_list.append(keypoints)

    return keypoints_list

In [11]:
label = 'A'  # Thay đổi tên nhãn theo ý bạn
keypoints_data = load_keypoints(label)

In [12]:
for idx, keypoints in enumerate(keypoints_data):
    
    print(f'Keypoints from image {idx+1} in label {label}:')
    
    print(keypoints)
    print('---')

Keypoints from image 1 in label A:
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.000